# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

#plottings
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use("ggplot")

In [2]:
df=pd.read_csv('homepage_actions.csv', header=0)
print(df.shape)
df.head()

(8188, 4)


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
#Your code here
clicks=set(df[df.action=='click']['id'].unique())
views=set (df[df.action=='view']['id'].unique() )

print(f'Total Number of people Viewed: {len(views)} ')
print(f'Total Number of people Viewed and Clicked: {len(clicks)} ')
print(f'Total Number of people Viewed and didnt click: {len(views-clicks)} ')
print(f'Total Number of people clicked and did not view: {len(clicks-views)} ')

Total Number of people Viewed: 6328 
Total Number of people Viewed and Clicked: 1860 
Total Number of people Viewed and didnt click: 4468 
Total Number of people clicked and did not view: 0 


In [4]:
pple=set( df.id.unique())
action=set( df[ df.group=='control']['id'].unique() )
exp= set( df[ df.group=='experiment']['id'].unique())


print(f'Number of Persons: {len(pple)}')
print(f'Number of Persons in control group: {len(action)}  ')
print(f'Number of Persons in experiment group: {len(exp)}')


Number of Persons: 6328
Number of Persons in control group: 3332  
Number of Persons in experiment group: 2996


In [5]:
# those who clicked and viewed per group
df.groupby(['group','action'])[['timestamp']].count()

timestamp
group      action           
control    click         932
           view         3332
experiment click         928
           view         2996

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
# those who viewed and clicked in both groups
df[ df.action=='click'].groupby(['group'])[['id']].count()

,id
group,
control,932
experiment,928


In [7]:
ctrl=df[df.group=='control'].pivot(index='id', columns='action', values='action')['click']
ctrl.replace({'click':1 },inplace=True)
ctrl.fillna(value=0, inplace=True)

exp=df[ df.group=='experiment'].pivot(index='id', columns='action', values='action')['click']
exp.replace({'click':1} , inplace=True)
exp.fillna(value=0, inplace=True)
exp

id
182988    0.0
183136    0.0
183141    1.0
183283    0.0
183389    0.0
         ... 
935382    0.0
935576    0.0
935742    1.0
936129    0.0
937139    1.0
Name: click, Length: 2996, dtype: float64

In [8]:
#Your code here

#Ho: average click rate in both webpages is same: mu(exp) =< mu(ctrl)
#Ha: Average click rate in the experiment webpage is more than the control webpage : mu(exp) > mu(ctrl)

t,p=stats.ttest_ind(exp,ctrl,equal_var=False, alternative='greater')
print(p)

if p<=0.01:
    print('Reject Ho')
else:
    print('Fail to reject Ho')

0.004466402814337102
Reject Ho


In [9]:
""" 
Since the pvalue is less than alpha(0.01), we reject and conclude given an alpha of 0.01 ,we have sufficient evidence
to say that the click rate if the experimental webpage is greater than that of the control webpage
"""

' \nSince the pvalue is less than alpha(0.01), we reject and conclude given an alpha of 0.01 ,we have sufficient evidence\nto say that the click rate if the experimental webpage is greater than that of the control webpage\n'

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [10]:
#Your code here
mean=ctrl.mean()
mean*len(exp)

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [11]:
#Your code here
actual_mean_cliks=exp.mean()*len(exp)
sd=np.sqrt(len(exp)*mean *(1-mean) )

print(sd)

z=(actual_mean_cliks- (mean*len(exp)) ) / sd
z

24.568547907005815


3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [12]:
#Your code here
pval=stats.norm.sf(z)   #1-stats.norm.cdf(z)
pval

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [13]:
"""
Yes, in both the test the pvalues are less than an alpha of 0.01, therefore in both scenarios we reject the null hyothesis 
and conclude that the experimental webpage has more click rate than the control webpage.
"""

'\nYes, in both the test the pvalues are less than an alpha of 0.01, therefore in both scenarios we reject the null hyothesis \nand conclude that the experimental webpage has more click rate than the control webpage.\n'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.